# Neural Networks

1. ### torch.nn -> Be constructed a Neural Network

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

2. ### A typical training procedure for a NN is:
    - Define the NN that has some learnable params (or Weights)
    - iterate over a dataset of Inputs
    - Process input through the network
    - Computer the loss (how far is the output from being correct)
    - Propagate gradients back into the network’s parameters
    - Update the weights of the network, typically using a simple update rule:
        - weight = weight - learning_rate * gradient
        
        
3. ### Other
    - x.view -> to resize or reshape a tensor
        - Ex:  = torch.randn(4, 4) y = x.view(16) z = x.view(-1, 8)  
            ##### -> torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])
            
        - #### torch.Tensor 
            - A multi-dimensional array with support for autograd operations like backward(). Also holds the gradient w.r.t. the tensor.
        - #### nn.Module 
            - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.
        - #### nn.Parameter 
            - A kind of Tensor, that is automatically registered as a parameter when assigned as an attribute to a Module.
        - #### autograd.Function 
            - Implements forward and backward definitions of an autograd operation. Every Tensor operation, creates at least a single Function node, that connects to functions that created a Tensor and encodes its history.         
    

# Define the network

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# Create Net Class
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square Conv Kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        
        # 6-input, 16-output, 5x5 Conv
        self.conv2 = nn.Conv2d(6, 16, 5) 
        
        #An affine operation: y = Wx + b nn.Linear(in, out)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # Reshape x to feed for Fully Connection layer
        x = x.view(-1, self.num_flat_features(x))
        print(x)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)
        
        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


### The learnable params of a model are returned 
    - net.parameters()

In [3]:
params = list(net.parameters())
print(len(params))

# Conv 1's weight
print(params[0].size()) 

10
torch.Size([6, 1, 5, 5])


In [4]:
input = torch.randn(1, 1, 32, 32)
print(input )
out = net(input)
print(out)

tensor([[[[-1.4484, -1.5084, -1.4917,  ...,  0.2543,  1.2232,  1.0744],
          [ 0.2502, -0.5536, -1.0087,  ...,  0.6849, -0.3433, -2.2479],
          [-0.0577, -0.9031, -1.5983,  ..., -0.7729, -0.4029,  0.5592],
          ...,
          [-1.4986, -0.4177, -0.5413,  ..., -0.4576,  0.4274,  1.3754],
          [-0.1447,  0.0334, -0.4283,  ..., -0.1088,  0.5749,  0.0719],
          [-1.0549,  1.4417,  0.3850,  ...,  1.0538, -0.1239,  0.6827]]]])
tensor([[ 0.4976,  0.4914,  0.4222,  0.6027,  0.3324,  0.3767,  0.5629,
          0.5684,  0.4425,  0.5847,  0.5089,  0.2855,  0.9547,  0.2774,
          0.3868,  0.5210,  0.3204,  0.5549,  0.4800,  0.3581,  0.6162,
          0.7754,  0.2235,  0.3810,  0.5247,  0.2522,  0.2928,  0.1359,
          0.0000,  0.1158,  0.1035,  0.2183,  0.2256,  0.3154,  0.1155,
          0.1764,  0.4686,  0.0176,  0.4547,  0.4918,  0.1044,  0.3748,
          0.0000,  0.1811,  0.3755,  0.4462,  0.3117,  0.2620,  0.4781,
          0.3049,  0.3504,  0.2577,  0.0000,  

#### Zero the gradient buffers of all parameters and backprops with random gradients:
    - torch.nn only supports mini-batches. The entire torch.nn package only supports inputs that are a mini-batch of samples, and not a single sample.

In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

# Loss Funtion
A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

In [6]:
output = net(input)

# a dummy targer
target = torch.arange(1, 11)
print(target)

target = target.view(1, -1)
print(target)

#Make it the same shape ouput
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor([[ 0.4976,  0.4914,  0.4222,  0.6027,  0.3324,  0.3767,  0.5629,
          0.5684,  0.4425,  0.5847,  0.5089,  0.2855,  0.9547,  0.2774,
          0.3868,  0.5210,  0.3204,  0.5549,  0.4800,  0.3581,  0.6162,
          0.7754,  0.2235,  0.3810,  0.5247,  0.2522,  0.2928,  0.1359,
          0.0000,  0.1158,  0.1035,  0.2183,  0.2256,  0.3154,  0.1155,
          0.1764,  0.4686,  0.0176,  0.4547,  0.4918,  0.1044,  0.3748,
          0.0000,  0.1811,  0.3755,  0.4462,  0.3117,  0.2620,  0.4781,
          0.3049,  0.3504,  0.2577,  0.0000,  0.0374,  0.4039,  0.4192,
          0.1697,  0.3427,  0.0000,  0.1019,  0.4100,  0.2589,  0.0000,
          0.5322,  0.4162,  0.2180,  0.2154,  0.1465,  0.0000,  0.0379,
          0.0616,  0.0000,  0.3915,  0.0509,  0.0000,  0.7016,  0.6925,
          0.9302,  0.6271,  0.8368,  1.0130,  0.9886,  0.7437,  0.8126,
          0.5872,  0.6298,  0.6498,  0.9592,  0.7379,  0.5643,  1.1002,
          0.7080,  0.7541,  0.7721,  0.6481,  0.8673,  0.8251,  

#  Backprop

In [7]:
# zeroes the gradient buffers of all params
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([ 0.,  0.,  0.,  0.,  0.,  0.])
conv1.bias.grad after backward
tensor([-0.0018,  0.1106, -0.0105, -0.0509, -0.0380, -0.0118])


# Update the weights
 
### Stochastic Gradient Descent (SGD):
        weight = weight - learning_rate * gradient

In [10]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

#  SGD
However, as use NN, want to use various different update rules as SGD

In [12]:
import torch.optim as optim

In [13]:
# Create your optimizer
optimizer = optim.SGD(net.parameters(), lr= 0.01)

# In training loop:
optimizer.zero_grad() # Zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

tensor([[ 0.4847,  0.4703,  0.4000,  0.5825,  0.3136,  0.3630,  0.5433,
          0.5407,  0.4286,  0.5630,  0.4807,  0.2699,  0.9264,  0.2530,
          0.3739,  0.4964,  0.2937,  0.5327,  0.4662,  0.3328,  0.5934,
          0.7545,  0.1964,  0.3512,  0.5065,  0.3052,  0.3521,  0.2007,
          0.0351,  0.1709,  0.1621,  0.2872,  0.2878,  0.3645,  0.1765,
          0.2361,  0.5216,  0.0845,  0.5150,  0.5490,  0.1754,  0.4478,
          0.0000,  0.2298,  0.4402,  0.5143,  0.3890,  0.3306,  0.5364,
          0.3649,  0.3201,  0.2181,  0.0000,  0.0065,  0.3663,  0.3863,
          0.1325,  0.3097,  0.0000,  0.0746,  0.3769,  0.2182,  0.0000,
          0.4910,  0.3860,  0.1884,  0.1710,  0.1164,  0.0000,  0.0000,
          0.0245,  0.0000,  0.3442,  0.0093,  0.0000,  0.7442,  0.7436,
          0.9840,  0.6688,  0.8794,  1.0516,  1.0383,  0.7922,  0.8648,
          0.6246,  0.6750,  0.7099,  1.0075,  0.7855,  0.6067,  1.1525,
          0.7542,  0.7983,  0.8161,  0.6916,  0.9079,  0.8716,  